In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy



from pytorch_CelebAToYearbook_cDCGAN import *

/Users/rajat/Box Sync/ImpGAN/pytorch_CelebAToYearbook_cDCGAN.py:3: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  Fil

In [2]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


dset = FolderWithImages(root='./yearbook/', input_transform=data_transforms['train'], target_transform=data_transforms['train']) #new to get indices??
dset.image_filenames.sort()
shuf = True #new
train_loader = torch.utils.data.DataLoader(dset, batch_size=batch_size, shuffle=shuf)

In [3]:
train_iter = iter(train_loader)
x,_,l = train_iter.next()

In [4]:
x.shape

torch.Size([64, 3, 224, 224])

In [9]:
import torchvision.models
from torchvision.models.resnet import model_urls

model_urls['resnet18'] = model_urls['resnet18'].replace('https://', 'http://')
model_conv = torchvision.models.resnet18(pretrained=True)



for param in model_conv.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, num_classes)


In [16]:
model_conv(x).shape

torch.Size([64, 22])

In [15]:
num_classes

22

In [21]:
y_onehot = torch.FloatTensor(64, 22)

# In your for loop
y_onehot.zero_()
y_onehot.scatter_(1, y_gender_[0:64].view(64,-1), 1)

tensor([[ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  ...,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        ...,
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]])

In [19]:
y_gender_[0:64].shape

torch.Size([64])

In [20]:
y_onehot.shape

torch.Size([64, 22])

In [ ]:
model = model_conv
optimizer = optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

criterion = nn.CrossEntropyLoss()

max_epoch = 30
bsize = 64

num_iter = 0
running_loss = 0.0
running_corrects = 0.0
for epoch in range(max_epoch):
    exp_lr_scheduler.step()
    while num_iter < len(train_loader):
        inputs,_,label_ids = train_iter.next()
        labels = y_gender_[label_ids]
        labels_onehot = torch.FloatTensor(bsize, num_classes)
        labels_onehot.zero_()
        labels_onehot.scatter_(1, labels.view(bsize,-1), 1)
        inputs,labels = Variable(inputs),Variable(labels)
        optimizer.zero_grad()
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels.long())
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        num_iter += 1
        print('num_iter: ' + str(num_iter) + ' loss: ' + str(running_loss/num_iter) + ' corrects: ' + str(running_corrects/num_iter))
    epoch_loss = running_loss / len(train_loader)
    epoch_acc = running_corrects.double() / len(train_loader)
    
    print('Epoch: ' + str(epoch) + ' ' + str(epoch_loss) + ' ' + str(epoch_acc))


#for epoch in range(max_epoch):

num_iter: 1 loss: 186.56092834472656 corrects: tensor(4)
num_iter: 2 loss: 186.12747192382812 corrects: tensor(5)
num_iter: 3 loss: 184.33776346842447 corrects: tensor(6)
num_iter: 4 loss: 185.45802688598633 corrects: tensor(5)
num_iter: 5 loss: 185.37782592773436 corrects: tensor(6)
num_iter: 6 loss: 184.8303197224935 corrects: tensor(6)
num_iter: 7 loss: 184.89770289829798 corrects: tensor(6)
num_iter: 8 loss: 185.2759494781494 corrects: tensor(6)
num_iter: 9 loss: 184.80131022135416 corrects: tensor(6)
num_iter: 10 loss: 185.07306671142578 corrects: tensor(5)
num_iter: 11 loss: 184.75882651589134 corrects: tensor(5)
num_iter: 12 loss: 184.68956883748373 corrects: tensor(5)
num_iter: 13 loss: 184.74062758225662 corrects: tensor(5)
num_iter: 14 loss: 185.0287628173828 corrects: tensor(5)
num_iter: 15 loss: 185.35140380859374 corrects: tensor(5)
num_iter: 16 loss: 185.2130069732666 corrects: tensor(5)
num_iter: 17 loss: 185.10089739631204 corrects: tensor(5)
num_iter: 18 loss: 185.0225

In [26]:
len(train_loader)

593

In [5]:
import torchvision.models
from torchvision.models.alexnet import model_urls

model_urls['alexnet'] = model_urls['alexnet'].replace('https://', 'http://')
model_conv = torchvision.models.alexnet(pretrained=True)



for param in model_conv.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, num_classes)


AttributeError: 'AlexNet' object has no attribute 'fc'

In [8]:
mod = torch.load('model_res18_12.pth',map_location=lambda storage, loc: storage)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/torch/serialization.py:367: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/torch/serialization.py:367: SourceChangeWarning: source code of class 'torch.nn.modules.batchnorm.BatchNorm2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/torch/serialization.py:367: SourceChangeWarning: source code of class 'torch.nn.modules.acti

RuntimeError: unexpected EOF. The file might be corrupted.

In [13]:
model_conv.load_state_dict(torch.load('model_test(1).pth',map_location=lambda storage, loc: storage))

In [15]:
model_conv = model_conv.eval()
outputs = model_conv(x)
_, preds = torch.max(outputs, 1)
labels = y_gender_[l]

In [16]:
torch.sum(preds == labels)

tensor(34)

In [21]:
data_transforms['train'](x)

TypeError: 'builtin_function_or_method' object is not subscriptable

In [25]:
ttr(x[1,:,:,:])

TypeError: 'builtin_function_or_method' object is not subscriptable

In [26]:
y = transforms.ToPILImage()(x[1,:,:])

In [28]:
xx = data_transforms['train'](y)

In [30]:
xx.shape

torch.Size([3, 224, 224])